# Naive predictors

Make naive predictions for the whole data set in order to find out scores for these predictors on Kaggle platform and because there is no parameters to train for the chosen naive predictors.

Import libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime  
from datetime import timedelta  
from sagemaker import get_execution_role


Read data

In [2]:
X = pd.read_pickle('../01_preprocessed_data/X.pkl')


In [3]:
role = get_execution_role()
bucket='sagemaker-eu-central-1-594657351600'
prefix = 'kaggle-m5/00_data'

data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'calendar.csv')
calendar = pd.read_csv(data_location)

data_location = 's3://{}/{}/{}'.format(bucket, prefix, 'sample_submission.csv')
submission =  pd.read_csv(data_location)

Determine the dates for prediction

In [4]:
calendar['date'] =  pd.to_datetime(calendar['date']) 
calendar.date[calendar['d'] == "d_1914"].iloc[0]

Timestamp('2016-04-25 00:00:00')

In [5]:
start_pred_date = calendar.date[calendar['d'] == "d_1914"].iloc[0]
days_to_predict = 28
dates_to_predict = [start_pred_date + timedelta(days=x)  for x in range(days_to_predict)]
dates_to_predict

[Timestamp('2016-04-25 00:00:00'),
 Timestamp('2016-04-26 00:00:00'),
 Timestamp('2016-04-27 00:00:00'),
 Timestamp('2016-04-28 00:00:00'),
 Timestamp('2016-04-29 00:00:00'),
 Timestamp('2016-04-30 00:00:00'),
 Timestamp('2016-05-01 00:00:00'),
 Timestamp('2016-05-02 00:00:00'),
 Timestamp('2016-05-03 00:00:00'),
 Timestamp('2016-05-04 00:00:00'),
 Timestamp('2016-05-05 00:00:00'),
 Timestamp('2016-05-06 00:00:00'),
 Timestamp('2016-05-07 00:00:00'),
 Timestamp('2016-05-08 00:00:00'),
 Timestamp('2016-05-09 00:00:00'),
 Timestamp('2016-05-10 00:00:00'),
 Timestamp('2016-05-11 00:00:00'),
 Timestamp('2016-05-12 00:00:00'),
 Timestamp('2016-05-13 00:00:00'),
 Timestamp('2016-05-14 00:00:00'),
 Timestamp('2016-05-15 00:00:00'),
 Timestamp('2016-05-16 00:00:00'),
 Timestamp('2016-05-17 00:00:00'),
 Timestamp('2016-05-18 00:00:00'),
 Timestamp('2016-05-19 00:00:00'),
 Timestamp('2016-05-20 00:00:00'),
 Timestamp('2016-05-21 00:00:00'),
 Timestamp('2016-05-22 00:00:00')]

## Sales 364 days ago

For each item take sales 364 days ago

In [6]:
dates_last_year = [x - timedelta(days=(364))  for x in dates_to_predict ]

In [7]:
naive_prediction_last_year = X[X.date.isin(dates_last_year)]

In [8]:
predictions = pd.pivot(naive_prediction_last_year[['id', 'date', 'demand']], index = 'id', 
                       columns = 'date', values = 'demand').reset_index()

In [9]:
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

Evaluation rows are all zeros

In [10]:
evaluation_rows = [row for row in submission['id'] if 'evaluation' in row] 
evaluation = submission[submission['id'].isin(evaluation_rows)]
evaluation.head()

id  F1  F2  F3  F4  F5  F6  F7  F8  F9  ...  \
30490  HOBBIES_1_001_CA_1_evaluation   0   0   0   0   0   0   0   0   0  ...   
30491  HOBBIES_1_002_CA_1_evaluation   0   0   0   0   0   0   0   0   0  ...   
30492  HOBBIES_1_003_CA_1_evaluation   0   0   0   0   0   0   0   0   0  ...   
30493  HOBBIES_1_004_CA_1_evaluation   0   0   0   0   0   0   0   0   0  ...   
30494  HOBBIES_1_005_CA_1_evaluation   0   0   0   0   0   0   0   0   0  ...   

       F19  F20  F21  F22  F23  F24  F25  F26  F27  F28  
30490    0    0    0    0    0    0    0    0    0    0  
30491    0    0    0    0    0    0    0    0    0    0  
30492    0    0    0    0    0    0    0    0    0    0  
30493    0    0    0    0    0    0    0    0    0    0  
30494    0    0    0    0    0    0    0    0    0    0  

[5 rows x 29 columns]

In [11]:
validation_rows = [row for row in submission['id'] if 'validation' in row] 
validation = submission[submission['id'].isin(validation_rows)]
validation = validation[['id']].merge(predictions, on = 'id', how = 'left')
validation.head()


id   F1   F2   F3   F4   F5   F6   F7   F8   F9  \
0  HOBBIES_1_001_CA_1_validation  1.0  0.0  0.0  0.0  1.0  1.0  2.0  0.0  0.0   
1  HOBBIES_1_002_CA_1_validation  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  1.0   
2  HOBBIES_1_003_CA_1_validation  0.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  1.0   
3  HOBBIES_1_004_CA_1_validation  0.0  2.0  1.0  2.0  1.0  6.0  7.0  0.0  2.0   
4  HOBBIES_1_005_CA_1_validation  2.0  1.0  2.0  0.0  2.0  3.0  4.0  2.0  0.0   

   ...  F19  F20  F21  F22  F23  F24  F25  F26  F27  F28  
0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1  ...  0.0  0.0  1.0  0.0  3.0  0.0  1.0  0.0  0.0  0.0  
2  ...  0.0  1.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0  
3  ...  2.0  6.0  5.0  2.0  1.0  0.0  1.0  3.0  4.0  2.0  
4  ...  0.0  0.0  2.0  0.0  0.0  0.0  0.0  0.0  0.0  3.0  

[5 rows x 29 columns]

In [12]:
final = pd.concat([validation, evaluation])
final = final.fillna(0)
final.to_csv("../04_submissions/naive_prediction_last_year.csv", index = False)

## Sales on the same weekday

For each item take the average sold on the same day of the week

Add weekday to X

In [13]:
weekly_means_df = X.groupby(['id','weekday'])['demand'].agg(['mean']).reset_index()

In [14]:
dates_to_predict_df = pd.DataFrame(dates_to_predict, columns =['date']) 
dates_to_predict_df = dates_to_predict_df.merge(calendar[['date', 'weekday']], on = 'date', how = 'left')

In [15]:
dates_to_predict_df.head()

date    weekday
0 2016-04-25     Monday
1 2016-04-26    Tuesday
2 2016-04-27  Wednesday
3 2016-04-28   Thursday
4 2016-04-29     Friday

In [16]:
naive_prediction_weekly=submission[['id']]

In [17]:
for i in range(0, 28):
    weekday = dates_to_predict_df.loc[i, 'weekday']
    weekly_means = weekly_means_df[weekly_means_df['weekday'] == weekday]
    weekly_means = weekly_means[['id', 'mean']]
    weekly_means = weekly_means.rename(columns={"mean": 'F'+str(i+1)})
    naive_prediction_weekly = naive_prediction_weekly.merge(weekly_means, on = 'id', how = 'left')

In [18]:
naive_prediction_weekly = naive_prediction_weekly.fillna(0)

In [19]:
naive_prediction_weekly.to_csv("../04_submissions/naive_prediction_weekly.csv", index = False)